In [1]:
from functools import partial, reduce
import os
from operator import or_, and_, contains

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH, ALL
from dash.exceptions import PreventUpdate
from dateutil.parser import parse
import django
from django.db.models import Q
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from toolz import keyfilter, valfilter, merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
from plotter.graph import *
# from plotter.views import *
# from plotter.forms import *
from utils import (
    rows, 
    columns, 
    eta_methods, 
    qlist,
    keygrab, 
    in_me, 
    particular_fields_search,
    pickitems,
    pickctx
)


In [2]:
app = dash.Dash()
CACHE_CONFIG = {
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': './.cache'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

# we're using this to create thread-safe global values.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.
# we could instead just memoize to memory using lru_cache or similar

cset = cache_set(cache)
cget = cache_get(cache)


In [3]:
fig = main_graph()
graph_function = scatter
spec_model = MSpec 
obs_model = MObs
# update this -- a buncha fields, both spec and obs
searchable_fields = ['group','formation','member']
# active queryset is explicitly stored in global cache
cset('queryset', spec_model.objects.all().prefetch_related("observation"))
x_inputs = [
    Input('filter-1-x', 'value'),
    Input('filter-2-x', 'value'),
    Input('filter-3-x', 'value'),
    Input('axis-option-x', 'value'),
]
y_inputs = [
    Input('filter-1-y', 'value'),
    Input('filter-2-y', 'value'),
    Input('filter-3-y', 'value'),
    Input('axis-option-y', 'value')
]

In [4]:
recalculate_graph

<function plotter.graph.recalculate_graph(*args, x_inputs, y_inputs, graph_function, cget, cset)>

In [5]:
# insert 'global' variables for this app into functions

recalculate_graph = partial(
    recalculate_graph, 
    x_inputs=x_inputs,
    y_inputs=y_inputs,
    cget=cget,
    cset=cset,
    graph_function = graph_function
)

update_model_field = partial(update_model_field, cget=cget)
update_queryset = partial(update_queryset, cget=cget, cset=cset, spec_model = spec_model)

In [6]:
# memoize functions that seem to want caching
# this isn't totally working because the local version isn't mostly what's
# actually being called. doesn't deal with checks in handle_search, etc

for function in [
    handle_search, 
    particular_fields_search,
    qlist,
    make_axis,
    update_model_field,
    update_queryset
    ]:
    function = cache.memoize()(function)

In [7]:
def field_drop(fields, element_id, index):
    """dropdown for field selection -- no special logic atm"""
    return dcc.Dropdown(
        id={'type':element_id, 'index':index},
        options=[{"label": field, "value": field} for field in fields],
    )


def model_options_drop(queryset, field, element_id, index):
    """
    dropdown for selecting search values for a specific field
    could end up getting unmanageable as a UI element
    """
    return dcc.Dropdown(
        id={'type':element_id, 'index':index},
        options={"label": "any", "value": "any"}, 
        multi=True
    )

def search_parameter_div(index, searchable_fields, cget):
    return html.Div(children = [
        field_drop(searchable_fields, 'field-search', index),
        model_options_drop(cget('queryset'), 'group', 'value-search', index)
        ])

In [8]:
app.layout = html.Div(children = [
    html.Div(children = [
        axis_value_drop('axis-option-x'),
        filter_drop(spec_model,'filter-1-x'),
        filter_drop(spec_model,'filter-3-x'),
        filter_drop(spec_model,'filter-2-x'),
    ]),
    html.Div(children = [
        axis_value_drop('axis-option-y'),
        filter_drop(spec_model,'filter-1-y'),
        filter_drop(spec_model,'filter-3-y'),
        filter_drop(spec_model,'filter-2-y'),
    ]),
    html.Div(id = 'calculation-state', style={"display":"none"}),
    html.Div(id = 'search-trigger', style={"display":"none"}),
    # this is initially empty and populated on page load
    html.Div(
        children = [],
        id = 'search-container'
    ),
    html.Button('add search parameter', id='add-param', n_clicks=0),
    html.Button(id='submit-search', children='Submit', n_clicks=0),
    html.Div(children = [fig], id = 'main-graph-container'),
    ])

In [9]:

def add_dropdown(n_clicks, children, searchable_fields, cget):
    children.append(search_parameter_div(n_clicks, searchable_fields, cget))
    return children

add_dropdown = partial(add_dropdown, cget=cget, searchable_fields=searchable_fields)

app.callback(
    Output('search-container', 'children'),
    [Input('add-param', 'n_clicks')],
    [State('search-container', 'children')])(add_dropdown)





<function dash.dash.Dash.callback.<locals>.wrap_func.<locals>.add_context(n_clicks, children, *, searchable_fields=['group', 'formation', 'member'], cget=<function cache_get.<locals>.cget at 0x7f78effe7670>)>

In [10]:
def handle_search(model, search_dict, field_dict):
    """
    dispatcher / manager. right now handles: 'any,' reference to parent
    need to supplement with non-string search functions.
    """
    # toss out 'any' entries -- they do not restrict the search
    search_dict = valfilter(lambda x: x != "any", search_dict)
    
    # if we have no constraints, return the entire data set
    if not search_dict:
        return model.objects.all()
    
    return particular_fields_search(model, search_dict, searchable_fields)


def update_queryset(n_clicks, fields, values, cget, cset, spec_model):
    """
    updates the spectra displayed in the graph view.
    
    we'd actually like to extend this to include fields from both spec
    and obs
    
    and multiple fields
    """
    # don't do anything on page load
    # or if a blank request is issued
    if not (fields and values):
        raise PreventUpdate

    search_dict = {
        field:value for field, value in zip(fields,values)
        if ((field is not None) and (value is not None))
    }
    
    # if every search parameter is blank, don't do anything
    if not search_dict:
        raise PreventUpdate
    
    # if the search parameters have changed,
    # make a new queryset and trigger graph update
    if handle_search(spec_model, search_dict, fields) != cget("queryset"):
        cset(
            "queryset",
            handle_search(
                spec_model, search_dict, fields
            ).prefetch_related("observation"),
        )
        return n_clicks

update_queryset = partial(update_queryset, cget=cget, cset=cset, spec_model = spec_model)

In [11]:
# register functions with app i/o

for axis in ['x', 'y']:
    app.callback(
        [
            Output('filter-1-'+axis, 'style'),
            Output('filter-2-'+axis, 'style'),
            Output('filter-3-'+axis, 'style'),
        ],
        [Input('axis-option-'+axis, 'value')]
    )(change_input_visibility)

app.callback(
    Output('main-graph', 'figure'),
    # maybe later add an explicit recalc button?
    [*x_inputs, *y_inputs, Input('search-trigger', 'value')],
)(recalculate_graph)


app.callback(
    Output({'type':'value-search', 'index':MATCH},'options'),
    [Input({'type':'field-search', 'index':MATCH},'value')],
    )(update_model_field)


app.callback(
        Output('search-trigger', 'value'),
        [Input('submit-search', 'n_clicks')],
        [State({'type': 'field-search', 'index': ALL}, 'value'),
        State({'type': 'value-search', 'index': ALL}, 'value')],
    )(update_queryset)


<function dash.dash.Dash.callback.<locals>.wrap_func.<locals>.add_context(n_clicks, fields, values, *, cget=<function cache_get.<locals>.cget at 0x7f78effe7670>, cset=<function cache_set.<locals>.cset at 0x7f78f001fca0>, spec_model=<class 'plotter.models.MSpec'>)>

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jul/2020 06:47:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:47:48] "POST /_dash-update-component HTTP/1.1" 204 -


{'formation': ['Murray']}


127.0.0.1 - - [14/Jul/2020 06:47:54] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [14/Jul/2020 06:47:55] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [14/Jul/2020 06:47:58] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [14/Jul/2020 06:48:02] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [14/Jul/2020 06:48:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:48:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:48:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:48:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:48:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:48:27] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [14/Jul/2020 06:48:27] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [14/Jul/2020 06:48:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.

{'formation': ['Murray'], 'member': ['Pettegrove Point']}


...

{'formation': ['Murray'], 'member': ['Pettegrove Point', 'Blunts Point']}


127.0.0.1 - - [14/Jul/2020 06:49:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:49:46] "POST /_dash-update-component HTTP/1.1" 200 -


{'formation': ['Murray'], 'member': ['Pettegrove Point']}


127.0.0.1 - - [14/Jul/2020 06:49:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 06:49:50] "POST /_dash-update-component HTTP/1.1" 200 -


{'formation': [], 'member': ['Pettegrove Point']}


In [ ]:
MSpec.objects.all()[1].observation.sol